In [ ]:
from langchain_ollama import OllamaLLM

In [ ]:
llm = OllamaLLM(model="llama3.1")
llm.invoke("The first man on the moon was ...")

In [ ]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
ISS Catering Services De Meern, Vishandel Sier AMSTELVEEN, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

### Read transaction data

In [ ]:
# Read the transactions csv file 
import pandas as pd
from finance_viz.utils.transaction_data_utils import PROJECT_ROOT

data_path = PROJECT_ROOT / "input_data" / "prepared_transaction_data.csv"

df = pd.read_csv(data_path)


print("entries of dataframe", df.shape[0])

df['company'] = df['company'].replace('R.S.C.', 'Radboud Sports Centrum')

print("data frame looks like")
df.head()
        


In [ ]:
# Get unique transactions in the Name / Description column
unique_transactions = df["company"].unique()

len(unique_transactions)

### Categorise bank transactions with Llama2

In [ ]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

In [ ]:
# Output validation
from pydantic import BaseModel, field_validator
from typing import List

# Validate response format - check if it actually contains hyphen ("-")
class ResponseChecks(BaseModel):
    data: List[str]

    @field_validator("data")
    def check(cls, value):
        for item in value:
            if len(item) > 0:
                assert "-" in item, "String does not contain hyphen."

# Test validation
ResponseChecks(data = ['Hello - World', 'Hello - there!'])

In [ ]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc..\
                           Categories should be less than 4 words. \
                           If you are unsure about a category, please respond with the closest appropriate match and do not provide explanations \
                           If you encounter a persons name in the categorization with the format Initial Lastname or First Name Last Name, classify it as IGNORE_PERSON. \
                           In case it is a bank, you can categorize it as IGNORE_SELF_TRANSFER.\
                           If you do not know the category at all, categorize it as Unknown. Please do not provide explanations." + transaction_names)
    response = response.split('\n')

    # Keep only the lines in between blank lines (removing the explaination lines at the beginning and end of the response)
    blank_indexes = [index for index in range(len(response)) if response[index] == '']
    if len(blank_indexes) == 1:
        response = response[(blank_indexes[0] + 1):]
    else:
        response = response[(blank_indexes[0] + 1) : blank_indexes[1]]

    # Print response and validate if it is in the correct format
    print(response)
    ResponseChecks(data = response)
    
    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [ ]:
# Test out the function
categorize_transactions('BCK*Hans Geveling Bloe, AH Station Nijmegen,Revolut Bank UAB',llm)

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()
max_tries = 7

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    # print(transaction_names)
    
    # Try and validate output, if it fails, try again for max_tries=7 times
    for j in range(1, max_tries):
        try:
            categories_df = categorize_transactions(transaction_names, llm)
            categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)
            
        except:
            if j < max_tries:
                continue
            else:
                raise Exception(f"Cannot categorise transactions indexes {i} to {i+1}.")
        break

In [ ]:
categories_df_all.head()

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [ ]:
# Remove the numbering eg "1. " from company column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'^\s*\d+\.\s*', '', regex=True)
categories_df_all.head()


In [ ]:
df_merged_with_categories = pd.merge(df, categories_df_all, left_on='company', right_on='Transaction', how='left')
df_merged_with_categories.head()


In [ ]:
# 1) Drop rows where 'company' is NaN
df_merged_with_categories = df_merged_with_categories.dropna(subset=["Category"])

# 2) Drop rows where 'company' is exactly 'Unknown'
df_merged_with_categories = df_merged_with_categories[df_merged_with_categories["Category"] != "Unknown"]

# 3) Drop rows where 'company' contains "IGNORE_"
df_merged_with_categories = df_merged_with_categories[~df_merged_with_categories["Category"].str.contains("IGNORE_PERSON|IGNORE_SELF_TRANSFER")]

# Post process categories: remove any other info inside the category column. i.e., sometimes the LLM will input 
# additional text despite or safe guards. Lke "Category (unnecessary text)"
df_merged_with_categories["Category"] = (
    df_merged_with_categories["Category"]
    .str.replace(r"\(.*?\)", "", regex=True)  # remove ( ... )
    .str.strip()  # remove extra whitespace
)

# check the categories
print(df_merged_with_categories['Category'].unique())

In [ ]:
data_new_path = PROJECT_ROOT / "input_data" / "transactions_categorized.csv"
df_merged_with_categories.to_csv(data_new_path, index=False)